In [1]:
from pyquery import PyQuery as pq
import requests
from tqdm import tqdm

import os

In [2]:
placeswanted = ["Paola", "S. Fili", "Cervicati"]

In [3]:
starturl = "http://www.onciario.beniculturali.it/?page_id=7#contenuto"
baseurl = "http://www.onciario.beniculturali.it"

In [4]:
downloaddir = "downloads/"
for placewanted in placeswanted:
    os.makedirs(downloaddir + placewanted, exist_ok=True)

In [5]:
r = requests.get(starturl)
starthtml = r.content

In [6]:
for placewanted in placeswanted:
    placeurl = None
    placeslinks = pq(starthtml)("a.ngg-album-desc")
    for placelink in pq(placeslinks)("a"):
        if pq(placelink).attr("title") == placewanted:
            placeurl = pq(placelink).attr("href")
    if not placeurl:
        print(f"Could not find an entry for {placewanted}")
    else:
        print(f"Trying to parse files for {placewanted}")
        r = requests.get(placeurl)
        placehtml = r.content
        lastpage = int(pq(placehtml)("a.page-numbers")[-1].text)    # Get last page number, convert to an actual number
        urlfragmentraw = pq(pq(placehtml)("a.page-numbers")[-1]).attr('href')  # Get the URL for the last page
        urlfragment = "=".join(urlfragmentraw.split("=")[:-1]) + "="     # Strip everything after the last =
        for pagenumber in tqdm(range(1, lastpage + 1)):
            r = requests.get(baseurl + urlfragment + str(pagenumber))
            pagehtml = r.content
            for box in pq(pagehtml)("div.ngg-gallery-thumbnail-box"):
                imgurl = pq(box)("a").attr("href")
                imgbasefilename = imgurl.split("/")[-1]     # Everything after the last slash is our filename
                targetfilename = downloaddir + placewanted + "/" + imgbasefilename
                if not os.path.exists(targetfilename):    # If we haven't gotten this file yet
                    r = requests.get(imgurl)
                    if r.status_code != 200:
                        print(f"Error downloading {imgurl}")
                    else:
                        with open(targetfilename, "wb") as outfile:
                            outfile.write(r.content)

Trying to parse files for Paola


 22%|██████████████████▍                                                                | 4/18 [05:54<20:40, 88.60s/it]


ConnectionError: HTTPConnectionPool(host='www.onciario.beniculturali.it', port=80): Max retries exceeded with url: /wp-content/gallery/onciario/Paola/0207(1).JPG (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230EF9919B0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))